In [8]:
# Celda 0: Inspección ultra-ligera del TXT gigante
# Objetivo:
#   - Ver tamaño real
#   - Leer primeras/últimas líneas sin cargar todo
#   - Inferir delimitador probable (coma, tab, ;, |)
#   - Detectar si parece CSV/TSV o JSONL

from pathlib import Path
import os

INVENTORY_PATH = r"C:\Quant\MT5_Data_Extraction\diagnostics_global\parquet_inventory_global_20251207_142248.txt"

p = Path(INVENTORY_PATH)
print("Existe:", p.exists())
if not p.exists():
    raise FileNotFoundError(f"No encuentro el archivo: {p}")

size_mb = p.stat().st_size / (1024**2)
print(f"Tamaño aproximado: {size_mb:,.2f} MB")

# Leer primeras líneas
print("\n--- HEAD (primeras 5 líneas) ---")
with open(p, "r", encoding="utf-8", errors="replace") as f:
    head_lines = [next(f) for _ in range(5)]
for i, line in enumerate(head_lines, 1):
    print(f"{i}: {line[:300].rstrip()}")

# Leer últimas líneas sin cargar todo (seek desde el final)
print("\n--- TAIL (últimas ~5 líneas) ---")
with open(p, "rb") as f:
    f.seek(0, os.SEEK_END)
    end = f.tell()
    block = 4096
    data = b""
    while end > 0 and data.count(b"\n") < 10:
        read_size = min(block, end)
        end -= read_size
        f.seek(end)
        data = f.read(read_size) + data
tail_text = data.decode("utf-8", errors="replace").splitlines()[-5:]
for i, line in enumerate(tail_text, 1):
    print(f"{i}: {line[:300].rstrip()}")

# Heurística de delimitador usando la primera línea "larga"
candidate = max(head_lines, key=len)
delims = [",", "\t", ";", "|"]
counts = {d: candidate.count(d) for d in delims}
print("\nConteo de delimitadores en línea candidata:", counts)

best_delim = max(counts, key=counts.get)
if counts[best_delim] == 0:
    print("⚠️ No se detecta delimitador claro. Podría ser JSONL u otro formato.")
else:
    print(f"Delimitador probable: {repr(best_delim)}")

print("\n✅ Celda 0 OK: inspección sin reventar RAM.")


Existe: True
Tamaño aproximado: 857.50 MB

--- HEAD (primeras 5 líneas) ---
1: PARQUET INVENTORY (GLOBAL)
2: ================================================================================
3: ROOT: C:\Quant\MT5_Data_Extraction
4: Generated: 20251207_142248
5: Total parquets: 1327124

--- TAIL (últimas ~5 líneas) ---
1:     - ok_spreadpct: Boolean
2:     - ok_all: Boolean
3: 
4: ================================================================================
5: FINISHED. Files with errors: 0

Conteo de delimitadores en línea candidata: {',': 0, '\t': 0, ';': 0, '|': 0}
⚠️ No se detecta delimitador claro. Podría ser JSONL u otro formato.

✅ Celda 0 OK: inspección sin reventar RAM.


In [9]:
# Celda 2: Convertir el inventario a Parquet ZSTD + versión lite
# Objetivo:
#   - Leer en modo eficiente
#   - Detectar columnas
#   - Guardar:
#       * inventory_sample.parquet (muestra grande)
#       * inventory_lite.parquet (solo columnas clave si existen)

import polars as pl
from pathlib import Path

INVENTORY_PATH = r"C:\Quant\MT5_Data_Extraction\diagnostics_global\parquet_inventory_global_20251207_142248.txt"
OUT_DIR = Path(r"C:\Quant\MT5_Data_Extraction\diagnostics_global")

# Si en Celda 0 salió otro delimitador, cámbialo aquí:
DELIM = ","  # o "\t" o ";" o "|"

OUT_SAMPLE_PARQUET = OUT_DIR / "inventory_sample_50k.parquet"
OUT_LITE_PARQUET   = OUT_DIR / "inventory_lite.parquet"

OUT_DIR.mkdir(parents=True, exist_ok=True)

print("Intentando leer 50k filas para inferir esquema...")
df = pl.read_csv(
    INVENTORY_PATH,
    separator=DELIM,
    infer_schema_length=5000,
    n_rows=50_000,
    ignore_errors=True,
)

print("Columnas detectadas:", df.columns)
print("Shape sample:", df.shape)

# Guardar sample parquet comprimido
df.write_parquet(str(OUT_SAMPLE_PARQUET), compression="zstd")
print("✅ Sample parquet:", OUT_SAMPLE_PARQUET, "| MB:", OUT_SAMPLE_PARQUET.stat().st_size / (1024**2))

# Heurística de columnas clave típicas de inventarios
lower_map = {c.lower(): c for c in df.columns}
preferred = [
    "file_path", "path", "parquet_path",
    "symbol", "dataset", "timeframe",
    "start", "end", "start_utc", "end_utc",
    "rows", "n_rows", "cols", "n_cols",
    "size", "size_bytes", "size_mb",
    "hash", "md5", "sha1",
]

lite_cols = []
for key in preferred:
    if key in lower_map:
        lite_cols.append(lower_map[key])

# Si no encontramos columnas estándar, al menos guardamos primeras 8
if not lite_cols:
    lite_cols = df.columns[: min(8, len(df.columns))]

df_lite = df.select(lite_cols).unique()

df_lite.write_parquet(str(OUT_LITE_PARQUET), compression="zstd")
print("✅ Lite parquet:", OUT_LITE_PARQUET, "| MB:", OUT_LITE_PARQUET.stat().st_size / (1024**2))
print("Lite cols:", lite_cols)

print("\n✅ Celda 2 OK.")


Intentando leer 50k filas para inferir esquema...


ComputeError: found more fields than defined in 'Schema'

Consider setting 'truncate_ragged_lines=True'.

In [ ]:
# Celda 3: Partir TXT gigante por tamaño aproximado
# Objetivo:
#   - Crear chunks para inspección o archivado
#   - No toca lógica estadística, solo gestión de archivo

from pathlib import Path

INVENTORY_PATH = r"C:\Quant\MT5_Data_Extraction\diagnostics_global\parquet_inventory_global_20251207_142248.txt"
OUT_DIR = Path(r"C:\Quant\MT5_Data_Extraction\diagnostics_global\inventory_chunks")

CHUNK_MB = 50

p = Path(INVENTORY_PATH)
OUT_DIR.mkdir(parents=True, exist_ok=True)

chunk_bytes = CHUNK_MB * 1024 * 1024
chunk_idx = 1
written = 0

print("Partiendo archivo...")
with open(p, "r", encoding="utf-8", errors="replace") as fin:
    fout = open(OUT_DIR / f"inventory_part_{chunk_idx:03d}.txt", "w", encoding="utf-8")
    for line in fin:
        fout.write(line)
        written += len(line.encode("utf-8", errors="replace"))

        if written >= chunk_bytes:
            fout.close()
            print(f"✅ Chunk {chunk_idx} creado")
            chunk_idx += 1
            written = 0
            fout = open(OUT_DIR / f"inventory_part_{chunk_idx:03d}.txt", "w", encoding="utf-8")

    fout.close()

print("Chunks en:", OUT_DIR)
print("\n✅ Celda 3 OK.")


Partiendo archivo...
✅ Chunk 1 creado
✅ Chunk 2 creado
✅ Chunk 3 creado
✅ Chunk 4 creado
✅ Chunk 5 creado
✅ Chunk 6 creado
✅ Chunk 7 creado
✅ Chunk 8 creado
✅ Chunk 9 creado
✅ Chunk 10 creado
✅ Chunk 11 creado
✅ Chunk 12 creado
✅ Chunk 13 creado
✅ Chunk 14 creado
✅ Chunk 15 creado
✅ Chunk 16 creado
Chunks en: C:\Quant\MT5_Data_Extraction\diagnostics_global\inventory_chunks

✅ Celda 3 OK.


In [ ]:
# Celda A: Extraer líneas críticas del TXT gigante a PARQUET (subible)
# -----------------------------------------------------------------------------
# Objetivo:
#   - Evitar el bloqueo por "demasiado contenido de texto"
#   - Extraer:
#       1) Todas las líneas que contengan ".parquet"
#       2) Un bloque pequeño del encabezado
#       3) Un bloque pequeño del cierre
#   - Guardar en:
#       * inventory_paths.parquet
#       * inventory_head_tail.parquet
#
# Resultado esperado:
#   - Archivos binarios livianos, fáciles de subir aquí.
# -----------------------------------------------------------------------------

from pathlib import Path
import re
import polars as pl
import os

INVENTORY_PATH = r"C:\Quant\MT5_Data_Extraction\diagnostics_global\parquet_inventory_global_20251207_142248.txt"
OUT_DIR = Path(r"C:\Quant\MT5_Data_Extraction\diagnostics_global\inventory_extracts")
OUT_DIR.mkdir(parents=True, exist_ok=True)

p = Path(INVENTORY_PATH)
if not p.exists():
    raise FileNotFoundError(f"No encuentro: {p}")

print("Existe:", p.exists())
print("Tamaño MB:", p.stat().st_size / (1024**2))

# --- Helper: leer head ---
HEAD_N = 200
head_lines = []
with open(p, "r", encoding="utf-8", errors="replace") as f:
    for _ in range(HEAD_N):
        line = f.readline()
        if not line:
            break
        head_lines.append(line.rstrip("\n"))

print(f"Head lines capturadas: {len(head_lines)}")

# --- Helper: leer tail sin cargar todo ---
def read_tail_lines(path: Path, approx_bytes=200_000):
    # lee un bloque al final y corta a líneas
    with open(path, "rb") as f:
        f.seek(0, os.SEEK_END)
        end = f.tell()
        start = max(0, end - approx_bytes)
        f.seek(start)
        data = f.read(end - start)
    text = data.decode("utf-8", errors="replace")
    lines = text.splitlines()
    # tomamos últimos 200 o menos
    return lines[-200:] if len(lines) > 200 else lines

tail_lines = read_tail_lines(p)
print(f"Tail lines capturadas: {len(tail_lines)}")

# --- Extraer líneas con ".parquet" ---
parquet_line_regex = re.compile(r"\.parquet", re.IGNORECASE)

parquet_lines = []
with open(p, "r", encoding="utf-8", errors="replace") as f:
    for line in f:
        if parquet_line_regex.search(line):
            parquet_lines.append(line.rstrip("\n"))

print(f"Líneas que contienen '.parquet': {len(parquet_lines):,}")

# --- Guardar a Parquet binario ---
paths_df = pl.DataFrame({"line": parquet_lines})
head_tail_df = pl.DataFrame({
    "section": ["head"] * len(head_lines) + ["tail"] * len(tail_lines),
    "line": head_lines + tail_lines
})

OUT_PATHS = OUT_DIR / "inventory_paths.parquet"
OUT_HEAD_TAIL = OUT_DIR / "inventory_head_tail.parquet"

paths_df.write_parquet(str(OUT_PATHS), compression="zstd")
head_tail_df.write_parquet(str(OUT_HEAD_TAIL), compression="zstd")

print("✅ Creado:", OUT_PATHS, "| MB:", OUT_PATHS.stat().st_size / (1024**2))
print("✅ Creado:", OUT_HEAD_TAIL, "| MB:", OUT_HEAD_TAIL.stat().st_size / (1024**2))

print("\n👉 Sube estos 2 archivos aquí:")
print("   1) inventory_paths.parquet")
print("   2) inventory_head_tail.parquet")
print("\n✅ Celda A OK.")


Existe: True
Tamaño MB: 857.4952621459961
Head lines capturadas: 200
Tail lines capturadas: 200
Líneas que contienen '.parquet': 2,654,248
✅ Creado: C:\Quant\MT5_Data_Extraction\diagnostics_global\inventory_extracts\inventory_paths.parquet | MB: 7.420282363891602
✅ Creado: C:\Quant\MT5_Data_Extraction\diagnostics_global\inventory_extracts\inventory_head_tail.parquet | MB: 0.0023736953735351562

👉 Sube estos 2 archivos aquí:
   1) inventory_paths.parquet
   2) inventory_head_tail.parquet

✅ Celda A OK.


In [12]:
# Celda C v3: Construcción profesional del inventario LITE (robusta + FIX regex ROOT)
# -----------------------------------------------------------------------------
# Objetivo:
#   - Tomar "inventory_paths.parquet" (líneas con .parquet)
#   - Extraer paths reales:
#       * abs_path: C:\...\*.parquet
#       * rel_path: prioriza formato "REL : xxx.parquet"
#   - Normalizar, deduplicar
#   - Generar:
#       1) inventory_lite.parquet
#       2) inventory_folder_stats.parquet
#       3) inventory_root_stats.parquet
#
# Fix crítico:
#   - Polars str.replace usa REGEX -> hay que escapar ROOT (Windows backslashes)
# -----------------------------------------------------------------------------

from pathlib import Path
import polars as pl
import re

IN_PATHS = r"C:\Quant\MT5_Data_Extraction\diagnostics_global\inventory_extracts\inventory_paths.parquet"
ROOT = r"C:\Quant\MT5_Data_Extraction"

OUT_DIR = Path(r"C:\Quant\MT5_Data_Extraction\diagnostics_global\inventory_extracts")
OUT_DIR.mkdir(parents=True, exist_ok=True)

OUT_LITE   = OUT_DIR / "inventory_lite.parquet"
OUT_FSTATS = OUT_DIR / "inventory_folder_stats.parquet"
OUT_RSTATS = OUT_DIR / "inventory_root_stats.parquet"

p = Path(IN_PATHS)
print("Existe inventory_paths.parquet:", p.exists())
if not p.exists():
    raise FileNotFoundError("No encuentro inventory_paths.parquet. Revisa la ruta.")

print("Leyendo inventory_paths.parquet...")
df_lines = pl.read_parquet(IN_PATHS)

if "line" not in df_lines.columns:
    raise RuntimeError("inventory_paths.parquet no tiene columna 'line'.")

print("Rows líneas:", df_lines.height)

# Normalización de líneas
df0 = df_lines.with_columns(
    pl.col("line")
      .cast(pl.Utf8)
      .str.replace_all("/", "\\")
      .alias("line_norm")
)

# Patrones
abs_pattern = r'([A-Za-z]:\\[^"\s]+?\.parquet)'

# PRIORIDAD: líneas del inventario parecen tener formato:
#   "REL : data\bulk_data\m5_raw\symbol=...\part=YYYYMMDD.parquet"
rel_pattern = r'REL\s*:\s*([^\r\n]+?\.parquet)'

# Fallback genérico:
any_parquet_pattern = r'([^"\s]+?\.parquet)'

df1 = df0.with_columns([
    pl.col("line_norm").str.extract(abs_pattern, 1).alias("abs_path"),
    pl.col("line_norm").str.extract(rel_pattern, 1).alias("rel_path_fmt"),
    pl.col("line_norm").str.extract(any_parquet_pattern, 1).alias("any_path"),
])

# Construir rel_path:
#   - si rel_path_fmt existe -> úsalo
#   - si no, usar any_path solo si parece path real (contiene "\")
df1 = df1.with_columns(
    pl.when(pl.col("rel_path_fmt").is_not_null())
      .then(pl.col("rel_path_fmt"))
      .when(pl.col("abs_path").is_null() & pl.col("any_path").str.contains(r"\\"))
      .then(pl.col("any_path"))
      .otherwise(None)
      .alias("rel_path")
)

# Normalizar ROOT
root_norm = ROOT.replace("/", "\\").rstrip("\\") + "\\"
root_pattern = re.escape(root_norm)  # ✅ FIX regex

# Construir parquet_path final:
df_paths = (
    df1
    .with_columns(
        pl.when(pl.col("abs_path").is_not_null())
          .then(pl.col("abs_path"))
          .when(pl.col("rel_path").is_not_null())
          .then(
              pl.concat_str([
                  pl.lit(root_norm),
                  pl.col("rel_path").str.replace_all(r"^[\\]+", "")
              ])
          )
          .otherwise(None)
          .alias("parquet_path")
    )
    .select(["line_norm", "abs_path", "rel_path", "parquet_path"])
)

# Diagnóstico duro
abs_count   = df_paths.filter(pl.col("abs_path").is_not_null()).height
rel_count   = df_paths.filter(pl.col("rel_path").is_not_null()).height
final_count = df_paths.filter(pl.col("parquet_path").is_not_null()).height

print(f"Abs detectados: {abs_count:,}")
print(f"Rel detectados: {rel_count:,}")
print(f"Paths finales válidos: {final_count:,}")

print("\nEjemplos rel_path (primeros 5):")
print(df_paths.filter(pl.col("rel_path").is_not_null()).select("rel_path").head(5))

# Quedarnos con paths válidos
df_valid = df_paths.select("parquet_path").drop_nulls()
print("Paths extraídos (bruto):", df_valid.height)

# Deduplicación
df_lite = df_valid.unique()
print("Paths únicos:", df_lite.height)

# Carpeta
df_lite = df_lite.with_columns(
    pl.col("parquet_path")
      .str.replace(r"\\[^\\]+\.parquet$", "")
      .alias("folder")
)

# Stats por carpeta
folder_stats = (
    df_lite
    .group_by("folder")
    .agg(pl.len().alias("parquet_count"))
    .sort("parquet_count", descending=True)
)

# Root bucket (usa replace con ROOT escapado)
df_lite = df_lite.with_columns(
    pl.when(pl.col("parquet_path").str.starts_with(root_norm))
      .then(
          pl.col("parquet_path")
            .str.replace(root_pattern, "")  # ✅ FIX: patrón seguro
            .str.split("\\")
            .list.first()
      )
      .otherwise(pl.lit("OUTSIDE_ROOT"))
      .alias("root_bucket")
)

root_stats = (
    df_lite
    .group_by("root_bucket")
    .agg(pl.len().alias("parquet_count"))
    .sort("parquet_count", descending=True)
)

# Guardar outputs
df_lite.select(["parquet_path", "folder", "root_bucket"]).write_parquet(str(OUT_LITE), compression="zstd")
folder_stats.write_parquet(str(OUT_FSTATS), compression="zstd")
root_stats.write_parquet(str(OUT_RSTATS), compression="zstd")

print(f"\n✅ OUTPUT lite   → {OUT_LITE} | rows={df_lite.height}")
print(f"✅ OUTPUT folder → {OUT_FSTATS} | rows={folder_stats.height}")
print(f"✅ OUTPUT root   → {OUT_RSTATS} | rows={root_stats.height}")

print("\nTop 10 carpetas por #parquets:")
print(folder_stats.head(10))

print("\nTop buckets bajo ROOT:")
print(root_stats)

print("\n✅ Celda C v3 OK.")


Existe inventory_paths.parquet: True
Leyendo inventory_paths.parquet...
Rows líneas: 2654248
Abs detectados: 0
Rel detectados: 1,327,124
Paths finales válidos: 1,327,124

Ejemplos rel_path (primeros 5):
shape: (5, 1)
┌─────────────────────────────────┐
│ rel_path                        │
│ ---                             │
│ str                             │
╞═════════════════════════════════╡
│ data\bulk_data\m5_raw\symbol=A… │
│ data\bulk_data\m5_raw\symbol=A… │
│ data\bulk_data\m5_raw\symbol=A… │
│ data\bulk_data\m5_raw\symbol=A… │
│ data\bulk_data\m5_raw\symbol=A… │
└─────────────────────────────────┘
Paths extraídos (bruto): 1327124
Paths únicos: 1327124

✅ OUTPUT lite   → C:\Quant\MT5_Data_Extraction\diagnostics_global\inventory_extracts\inventory_lite.parquet | rows=1327124
✅ OUTPUT folder → C:\Quant\MT5_Data_Extraction\diagnostics_global\inventory_extracts\inventory_folder_stats.parquet | rows=48803
✅ OUTPUT root   → C:\Quant\MT5_Data_Extraction\diagnostics_global\inventory_ext

In [13]:
# Celda D: Exportar resumen ultra pequeño para subir al chat
# -----------------------------------------------------------------------------
# Objetivo:
#   - Crear CSV muy liviano con las 200 carpetas más grandes
#   - Crear TXT diagnóstico mínimo
# -----------------------------------------------------------------------------

from pathlib import Path
import polars as pl

OUT_DIR = Path(r"C:\Quant\MT5_Data_Extraction\diagnostics_global\inventory_extracts")

IN_FSTATS = OUT_DIR / "inventory_folder_stats.parquet"
IN_RSTATS = OUT_DIR / "inventory_root_stats.parquet"

OUT_CSV_TOP200 = OUT_DIR / "inventory_folder_stats_top200.csv"
OUT_TXT_MINI = OUT_DIR / "inventory_inventory_mini_report.txt"

if not IN_FSTATS.exists():
    raise FileNotFoundError("No encuentro inventory_folder_stats.parquet. Ejecuta Celda C primero.")

folder_stats = pl.read_parquet(str(IN_FSTATS))
root_stats = pl.read_parquet(str(IN_RSTATS)) if IN_RSTATS.exists() else None

top200 = folder_stats.head(200)
top200.write_csv(str(OUT_CSV_TOP200))

lines = []
lines.append("INVENTORY MINI REPORT")
lines.append("=" * 80)
lines.append(f"folders_total = {folder_stats.height}")
lines.append(f"top200_csv    = {str(OUT_CSV_TOP200)}")

if root_stats is not None:
    lines.append("")
    lines.append("ROOT BUCKETS:")
    for row in root_stats.iter_rows(named=True):
        lines.append(f"  - {row['root_bucket']}: {row['parquet_count']}")

OUT_TXT_MINI.write_text("\n".join(lines), encoding="utf-8")

print("✅ Creado:", OUT_CSV_TOP200, "| KB:", OUT_CSV_TOP200.stat().st_size / 1024)
print("✅ Creado:", OUT_TXT_MINI, "| KB:", OUT_TXT_MINI.stat().st_size / 1024)

print("\n👉 Sube estos dos archivos aquí:")
print("   1) inventory_folder_stats_top200.csv")
print("   2) inventory_inventory_mini_report.txt")

print("\n✅ Celda D OK.")


✅ Creado: C:\Quant\MT5_Data_Extraction\diagnostics_global\inventory_extracts\inventory_folder_stats_top200.csv | KB: 23.9326171875
✅ Creado: C:\Quant\MT5_Data_Extraction\diagnostics_global\inventory_extracts\inventory_inventory_mini_report.txt | KB: 0.3544921875

👉 Sube estos dos archivos aquí:
   1) inventory_folder_stats_top200.csv
   2) inventory_inventory_mini_report.txt

✅ Celda D OK.


In [1]:
# Celda F: Breakdown profesional de inventario bajo ROOT/data
# -----------------------------------------------------------------------------
# Objetivo:
#   - Usar inventory_lite.parquet
#   - Calcular conteos por:
#       * nivel1 bajo ROOT (root_bucket ya lo tienes, pero lo recalculamos seguro)
#       * nivel1 y nivel2 específicamente dentro de "data"
#   - Generar CSV pequeños top50 para subir
# -----------------------------------------------------------------------------

from pathlib import Path
import polars as pl
import re

IN_LITE = r"C:\Quant\MT5_Data_Extraction\diagnostics_global\inventory_extracts\inventory_lite.parquet"
ROOT = r"C:\Quant\MT5_Data_Extraction"

OUT_DIR = Path(r"C:\Quant\MT5_Data_Extraction\diagnostics_global\inventory_extracts")
OUT_DIR.mkdir(parents=True, exist_ok=True)

OUT_DATA_L1 = OUT_DIR / "inventory_data_level1_stats.parquet"
OUT_DATA_L2 = OUT_DIR / "inventory_data_level2_stats.parquet"
OUT_DATA_L1_CSV = OUT_DIR / "inventory_data_level1_top50.csv"
OUT_DATA_L2_CSV = OUT_DIR / "inventory_data_level2_top50.csv"

p = Path(IN_LITE)
print("Existe inventory_lite.parquet:", p.exists())
if not p.exists():
    raise FileNotFoundError("No encuentro inventory_lite.parquet. Ejecuta Celda C v3 primero.")

df = pl.read_parquet(IN_LITE)
print("Rows inventory_lite:", df.height)

root_norm = ROOT.replace("/", "\\").rstrip("\\") + "\\"
root_pattern = re.escape(root_norm)

# Extraer path relativo bajo ROOT
df = df.with_columns(
    pl.when(pl.col("parquet_path").str.starts_with(root_norm))
      .then(pl.col("parquet_path").str.replace(root_pattern, ""))
      .otherwise(None)
      .alias("rel_under_root")
)

# Nivel 1 bajo ROOT
df = df.with_columns(
    pl.col("rel_under_root")
      .str.split("\\")
      .list.first()
      .alias("level1")
)

root_l1_stats = (
    df.filter(pl.col("level1").is_not_null())
      .group_by("level1")
      .agg(pl.len().alias("parquet_count"))
      .sort("parquet_count", descending=True)
)

print("\nROOT level1 stats:")
print(root_l1_stats)

# Filtrar solo data
df_data = df.filter(pl.col("level1") == "data").with_columns(
    pl.col("rel_under_root")
      .str.replace(r"^data\\", "")
      .alias("rel_under_data")
)

# Nivel 1 dentro de data
df_data = df_data.with_columns(
    pl.col("rel_under_data").str.split("\\").list.first().alias("data_level1")
)

data_l1_stats = (
    df_data.group_by("data_level1")
           .agg(pl.len().alias("parquet_count"))
           .sort("parquet_count", descending=True)
)

# Nivel 2 dentro de data
df_data = df_data.with_columns(
    pl.when(pl.col("rel_under_data").str.contains(r"\\"))
      .then(
          pl.col("rel_under_data").str.split("\\").list.slice(0, 2).list.join("\\")
      )
      .otherwise(pl.col("data_level1"))
      .alias("data_level2")
)

data_l2_stats = (
    df_data.group_by("data_level2")
           .agg(pl.len().alias("parquet_count"))
           .sort("parquet_count", descending=True)
)

# Guardar
data_l1_stats.write_parquet(str(OUT_DATA_L1), compression="zstd")
data_l2_stats.write_parquet(str(OUT_DATA_L2), compression="zstd")

# CSV top50 subibles
data_l1_stats.head(50).write_csv(str(OUT_DATA_L1_CSV))
data_l2_stats.head(50).write_csv(str(OUT_DATA_L2_CSV))

print("\n✅ OUTPUT:", OUT_DATA_L1)
print("✅ OUTPUT:", OUT_DATA_L2)
print("✅ CSV TOP50:", OUT_DATA_L1_CSV)
print("✅ CSV TOP50:", OUT_DATA_L2_CSV)

print("\nTop 10 data_level1:")
print(data_l1_stats.head(10))

print("\nTop 10 data_level2:")
print(data_l2_stats.head(10))

print("\n✅ Celda F OK.")


Existe inventory_lite.parquet: True
Rows inventory_lite: 1327124

ROOT level1 stats:
shape: (5, 2)
┌──────────────────────────┬───────────────┐
│ level1                   ┆ parquet_count │
│ ---                      ┆ ---           │
│ str                      ┆ u32           │
╞══════════════════════════╪═══════════════╡
│ data                     ┆ 1326899       │
│ venv1                    ┆ 210           │
│ outputs                  ┆ 13            │
│ respaldo                 ┆ 1             │
│ filtered_symbols.parquet ┆ 1             │
└──────────────────────────┴───────────────┘

✅ OUTPUT: C:\Quant\MT5_Data_Extraction\diagnostics_global\inventory_extracts\inventory_data_level1_stats.parquet
✅ OUTPUT: C:\Quant\MT5_Data_Extraction\diagnostics_global\inventory_extracts\inventory_data_level2_stats.parquet
✅ CSV TOP50: C:\Quant\MT5_Data_Extraction\diagnostics_global\inventory_extracts\inventory_data_level1_top50.csv
✅ CSV TOP50: C:\Quant\MT5_Data_Extraction\diagnostics_global\invent

In [2]:
# Celda G: Construir inventario limpio SOLO de data (trading-only)
# -----------------------------------------------------------------------------
# Objetivo:
#   - Eliminar ruido accidental del inventario global:
#       venv1, outputs, respaldo, archivos en raíz
#   - Guardar:
#       inventory_lite_data_only.parquet
# -----------------------------------------------------------------------------

from pathlib import Path
import polars as pl

IN_LITE = r"C:\Quant\MT5_Data_Extraction\diagnostics_global\inventory_extracts\inventory_lite.parquet"
OUT_DIR = Path(r"C:\Quant\MT5_Data_Extraction\diagnostics_global\inventory_extracts")
OUT_DATA_ONLY = OUT_DIR / "inventory_lite_data_only.parquet"

df = pl.read_parquet(IN_LITE)

df_data = df.filter(
    pl.col("parquet_path").str.contains(r"\\data\\")
)

df_data.write_parquet(str(OUT_DATA_ONLY), compression="zstd")

print("Rows total:", df.height)
print("Rows data-only:", df_data.height)
print("✅ OUTPUT:", OUT_DATA_ONLY)
print("\n✅ Celda G OK.")


Rows total: 1327124
Rows data-only: 1327109
✅ OUTPUT: C:\Quant\MT5_Data_Extraction\diagnostics_global\inventory_extracts\inventory_lite_data_only.parquet

✅ Celda G OK.
